# Caffe Nets, Layers, and Blobs

# 1. caffe?

* 많이 쓰이고 있는 Deep learning framework ( deep learning을 위한 C++/ CUDA architecture )
* reference Model (http://arxiv.org/pdf/1311.2524v1.pdf)
* Berkely Vision and Learning Center에서 개발

* Expression - 설정 값이 따로 hardcoding 되어 있지 않기 때문에 configuration 변경 만으로도 Model과 최적화에 대한 설계가 쉽다.
* Speed - GPU를 통한 병렬 연산을 통해 속도가 빠르다.
* Extensible code - 수많은 개발자들이 참여하고 있다.
* powerful community

## 1.1 How to Install (for Ubuntu)

http://yujuwon.tistory.com/entry/DeepLearning-%EC%9A%B0%EB%B6%84%ED%88%AC-1404%EC%97%90-Caffe-%EC%84%A4%EC%B9%98-%ED%95%98%EA%B8%B0

## 1.2 Nets, Layers, and Blobs

* Caffe 모델의 구조

## 1.2.1 Blobs

* 실제 데이터를 Caffe에서 알맞게 처리할 수 있도록 한 번 wrapping하는 역할.
* CPU와 GPU 간의 동기화를 지원하는 역할.
* lazily allocate memory
* 통합 메모리 인터페이스를 제공하고 image batch, 모델 parameter, 최적화 계수 등을 저장하는데 사용된다.

* data blob의 경우 Number, Channel, Height, Width로 구성된 4차원 배열
* Number는 data의 batch size를 의미하고 Channel은 pixel의 channel을 의미 (ex RGB = 3 channel)

* parameter blob은 layer의 구성과 타입에 따라 다양함.
* ex) 3개의 입력 blob을 가진 11x11 크기의 96개의 filter를 가진 convolution layer의 경우에는 blob 크기는 96 x 3 x 11 x 3
* ex) 1000개의 output을 가지고 1024개의 input channel을 가지는 fully-connected layer의 경우에는 blob 크기가 1 x 1 x 1000 x 1024

<img src="http://caffe.berkeleyvision.org/tutorial/fig/layer.jpg" />

## 1.2.1.1 Implementation Details

* CPU와 GPU상에 저장되는 데이터를 접근하는 방법
* const way
const Dtype* cpu_data() const;

* mutable way
Dtype* mutable_cpu_data();

* 이렇게 설계한 이유 - Blob 내부에서는 CPU와 GPU 사이의 동기화를 위해 SyncedMem이란 class를 사용하는데, CPU와 GPU 간의 data copy가 일어날 때마다 SyncedMem에서는 이를 알고 있어야 한다. 그래서 반드시 포인터를 가져오기 위한 함수를 호출하는 것을 장려하고, 직접 포인터 변수를 가져와서 변경하는 것은 금지하고 있음.

In [ ]:
// Assuming that data are on the CPU initially, and we have a blob.
const Dtype* foo;
Dtype* bar;
foo = blob.gpu_data(); // data copied cpu->gpu.
foo = blob.cpu_data(); // no data copied since both have up-to-date contents.
bar = blob.mutable_gpu_data(); // no data copied.
// ... some operations ...
bar = blob.mutable_gpu_data(); // no data copied when we are still on GPU.
foo = blob.cpu_data(); // data copied gpu->cpu, since the gpu side has modified the data
foo = blob.gpu_data(); // no data copied since both have up-to-date contents
bar = blob.mutable_cpu_data(); // still no data copied.
bar = blob.mutable_gpu_data(); // data copied cpu->gpu.
bar = blob.mutable_cpu_data(); // data copied gpu->cpu.

https://github.com/BVLC/caffe/blob/master/src/caffe/proto/caffe.proto

# 1.2.2 Layer

* filter, pool, rectified-linear, sigmoid ....
* http://caffe.berkeleyvision.org/tutorial/layers.html

In [ ]:
name: "conv1"
type: CONVOLUTION
bottom: "data"
top: "conv1"
convolution_param {
    num_output: 20
    kernel_size: 5
    stride: 1
    weight_filler{
        type: "xavier"
    }
}

* Layer 에서는 3가지의 중요한 계산 단계를 지닌다.
* Setup : layer와 연결 관계등을 초기화 한다.(model initialization에 한 번)
* Forward : 주어진 input으로부터 output을 계산하고 top으로 전달한다.
* Backward : output 결과에 따라 다시 input으로 전달한다.(오차 수정)

## 1.2.3 Net

* DAG(directed acyclic graph) 로 연결된 layer들의 집합.
* caffe에서는 정의를 통해 net을 생성하고 체크할 수 있다.

<img src="http://caffe.berkeleyvision.org/tutorial/fig/logreg.jpg" />

In [ ]:
name: "LogReg"
layer {
    name: "mnist"
    type: DATA
    top: "data"
    top: "label"
    data_param {
        source: "input_leveldb"
        batch_size: 64
    }
}
layer {
    name: "ip"
    type: INNER_PRODUCT
    bottom: "data"
    top: "ip"
    inner_product_param{
        num_output: 2
    }
}
layers {
    name: "loss"
    type: SOFTMAX_LOSS
    bottom: "ip"
    bottom: "label"
    top: "loss"
}

In [9]:
!python /home/konan/caffe/python/classify.py --print_results /home/konan/caffe/examples/images/cat.jpg foo

CPU mode
I0511 16:12:12.324367 153330 net.cpp:42] Initializing net from parameters: 
name: "CaffeNet"
input: "data"
input_dim: 10
input_dim: 3
input_dim: 227
input_dim: 227
state {
  phase: TEST
}
layer {
  name: "conv1"
  type: "Convolution"
  bottom: "data"
  top: "conv1"
  convolution_param {
    num_output: 96
    kernel_size: 11
    stride: 4
  }
}
layer {
  name: "relu1"
  type: "ReLU"
  bottom: "conv1"
  top: "conv1"
}
layer {
  name: "pool1"
  type: "Pooling"
  bottom: "conv1"
  top: "pool1"
  pooling_param {
    pool: MAX
    kernel_size: 3
    stride: 2
  }
}
layer {
  name: "norm1"
  type: "LRN"
  bottom: "pool1"
  top: "norm1"
  lrn_param {
    local_size: 5
    alpha: 0.0001
    beta: 0.75
  }
}
layer {
  name: "conv2"
  type: "Convolution"
  bottom: "norm1"
  top: "conv2"
  convolution_param {
    num_output: 256
    pad: 2
    kernel_size: 5
    group: 2
  }
}
layer {
  name: "relu2"
  type: "ReLU"
  bottom: "conv2"
  top: "conv2"
}
layer {
  name: "pool2"
  type: "Pooli